## 1. Import thư viện

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn import linear_model
from operator import itemgetter
import math

## 2. Load dữ liệu

In [2]:
movies = pd.DataFrame.from_csv('ml-1m/movies.csv')
movies.info()
movies.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie_id    3883 non-null int64
title       3883 non-null object
genres      3883 non-null object
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.DataFrame.from_csv('ml-1m/ratings.csv')
n_ratings = ratings.shape[0]
print 'Number of ratings: ', n_ratings
ratings.info()
ratings.head()

Number of ratings:  1000209
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 3 columns):
user_id     1000209 non-null int64
movie_id    1000209 non-null int64
rating      1000209 non-null int64
dtypes: int64(3)
memory usage: 30.5 MB


,user_id,movie_id,rating
0,1,1177,5
1,1,656,3
2,1,903,3
3,1,3340,4
4,1,2287,5


## 3. Tiền xử lý dữ liệu

### a. Dữ liệu movies

Cần chuyển thuộc tính genre thành ma trận one hot

In [4]:
def split_genre(genre):
    return genre.split('|')

def preprocessing_and_create_movie_feature(movies):
    spl_genre = movies['genres'].apply(split_genre).tolist()
    genre_cols = []
    for row in spl_genre:
        genre_cols.extend(row)
    genre_cols = list(set(genre_cols))
    genre_one_hot = []
    for row in spl_genre:
        row_one_hot = [0 if g not in row else 1 for g in genre_cols]
        genre_one_hot.append(row_one_hot)
    data = pd.DataFrame(genre_one_hot, columns=genre_cols)
    return data

In [5]:
movies_feature = preprocessing_and_create_movie_feature(movies)
movies_feature.head()

,Mystery,Drama,Sci-Fi,Fantasy,Horror,Film-Noir,Crime,Romance,Children's,Musical,Animation,Adventure,Action,Comedy,Documentary,War,Thriller,Western
0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [6]:
movie_feature = movies_feature.values
movie_feature.shape # [n_movies, n_feature]

(3883L, 18L)

### b. Dữ liệu ratings

Chia dữ liệu ratings thành train và test

In [7]:
ratings_matrix = ratings.values # convert to numpy array
ratings_matrix.shape

(1000209L, 3L)

In [8]:
n_users = int(np.max(ratings_matrix[:, 0]))
n_movies = int(np.max(ratings_matrix[:, 1]))
print 'Number of users: ', n_users
print 'Number of movies: ', n_movies

Number of users:  6040
Number of movies:  3883


In [9]:
# hàm chia dữ liệu thành các tập train, validation, test
def train_vali_test_split(data, train_size=0.6, vali_size=0.2):
    rnd_idxs = range(data.shape[0]) # Random indexes    
    np.random.shuffle(rnd_idxs)
    n_train = int(train_size*data.shape[0])
    n_vali = int(vali_size*data.shape[0])
    train_data = data[rnd_idxs[0:n_train]]
    vali_data = data[rnd_idxs[n_train:n_train+n_vali]]
    test_data = data[rnd_idxs[n_train+n_vali:]]
    return (train_data, vali_data, test_data)

In [10]:
# chỉ số của movie và user nên bắt đầu từ 0
ratings_matrix[:, :2] -= 1
train_ratings, vali_ratings, test_ratings = train_vali_test_split(ratings_matrix, train_size=0.6, vali_size=0.2)
print train_ratings.shape
print vali_ratings.shape
print test_ratings.shape

(600125L, 3L)
(200041L, 3L)
(200043L, 3L)


## 4. Mô hình content-based

### a. Hàm chuẩn hóa ma trận movie feature
Sử dụng l2 normalize để chuẩn hóa từng vector trong ma trận movie feature

In [11]:
def l2_normalizer(vec):
    denom = np.sum([el**2 for el in vec])
    return [(el / math.sqrt(denom)) for el in vec]

def normalize_feature(matrix):
    matrix_l2 = []
    for vec in matrix:
        matrix_l2.append(l2_normalizer(vec))
    return np.array(matrix_l2)

### b. Xây dựng mô hình

Mỗi dòng của movie_feature tương ứng với feature vector của một bộ phim. Với mỗi user, chúng ta cần lọc ra danh sách movies và ratings mà user đó đã rated

In [12]:
def get_movies_rated_by_user(ratings_matrix, user_id):
    # ratings_matrix[:,0] # cột user_id 
    rated = np.where(ratings_matrix[:,0] == user_id)[0] # tìm những hàng trong ratings được rate bởi user
    movie_ids = ratings_matrix[rated, 1].astype(np.int32) # danh sách movie id
    ratings = ratings_matrix[rated, 2] # và những ratings tương ứng
    return (movie_ids, ratings)

Để huấn luyện ta sử dụng ridge regression

In [13]:
def train_ridge_regression(train_ratings, X, n_users, learning_rate=0.1):
    """
    parameter:
        train_ratings: ma trận train ratings
        X: ma trận movie feature (đã chuẩn hóa hoặc chưa chuẩn hóa)
        n_users: number of users
        learning_rate: hệ số học
    return:
        W: ma trận vector output
    """
    # khởi tạo W, mỗi dòng của w là một vector ứng với mỗi user
    W = np.zeros((X.shape[1], n_users)) # init W shape[d + 1, n_users]
    for user_id in range(n_users):    
        movie_ids, ratings = get_movies_rated_by_user(train_ratings, user_id)
        model = Ridge(alpha=learning_rate, fit_intercept=False)    
        X_rated = X[movie_ids, :] # Danh sách feature vector của những movie đã rated
        try:
            model.fit(X_rated, ratings)
            W[:, user_id] = model.coef_
        except ValueError:
            continue
    return W

Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi. Lỗi được tính là hiệu của _true rating_ và _predicted rating_:

In [14]:
def RMSE(ratings_matrix, Y_pred):
    sum_err = 0
    count = 0
    for user_id in xrange(n_users):
        movie_ids, ratings = get_movies_rated_by_user(ratings_matrix, user_id)
        pred_ratings = Y_pred[movie_ids, user_id]
        err = ratings - pred_ratings
        sum_err += (err*err).sum(axis = 0)
        count += err.size 
    return math.sqrt(sum_err/count)

### c. Huấn luyện mô hình

In [15]:
X_nn = np.hstack([np.ones((len(movie_feature), 1)), movie_feature]) # not normalize
X_n = normalize_feature(movie_feature)
X_n = np.hstack([np.ones((len(X_n), 1)), X_n]) # normalized

#### Thử nghiệm so sánh ảnh hưởng của việc chuẩn hóa ma trận movie feature đến quá trình học. Cố định learning_rate=0.1

In [16]:
# TH1: không chuẩn hóa movie feature, learning_rate=0.1
W_1 = train_ridge_regression(train_ratings, X_nn, n_users) 
Y_pred_1 = X_nn.dot(W_1)
# tính độ lỗi
train_err_1 = RMSE(train_ratings, Y_pred_1)
vali_err_1 = RMSE(vali_ratings, Y_pred_1)
print 'RMSE for train: ', train_err_1
print 'RMSE for validation:  ', vali_err_1

RMSE for train:  0.90771385821
RMSE for validation:   1.07209087635


In [17]:
# TH2: chuẩn hóa movie feature, learning_rate=0.1
W_2 = train_ridge_regression(train_ratings, X_n, n_users)
Y_pred_2 = X_n.dot(W_2)
# tính độ lỗi
train_err_2 = RMSE(train_ratings, Y_pred_2)
vali_err_2 = RMSE(vali_ratings, Y_pred_2)
print 'RMSE for train: ', train_err_2
print 'RMSE for validation:  ', vali_err_2

RMSE for train:  0.908388858301
RMSE for validation:   1.05498628929


Từ thử nghiệm ta thấy chuẩn hóa ma trận movie feature cho kết quả tốt hơn. Ta sẽ sử dụng chuẩn hóa movie feature cho thử nghiệm tiếp theo.

#### Thử nghiệm so sánh ảnh hưởng của learning_rate đến quá trình học.
Thử nghiệm lần lượt với learning_rate=0.5, 0.1, 0.05

In [18]:
# learning_rate=0.5
W_3 = train_ridge_regression(train_ratings, X_n, n_users, learning_rate=0.5)
Y_pred_3 = X_n.dot(W_3) 
# tính độ lỗi
train_err_3 = RMSE(train_ratings, Y_pred_3)
vali_err_3 = RMSE(vali_ratings, Y_pred_3)
print 'RMSE for train: ', train_err_3
print 'RMSE for validation:  ', vali_err_3

RMSE for train:  0.918887996762
RMSE for validation:   1.03175004251


In [19]:
# learning_rate=0.1
W_4 = train_ridge_regression(train_ratings, X_n, n_users, learning_rate=0.1)
Y_pred_4 = X_n.dot(W_4) 
# tính độ lỗi
train_err_4 = RMSE(train_ratings, Y_pred_4)
vali_err_4 = RMSE(vali_ratings, Y_pred_4)
print 'RMSE for train: ', train_err_4
print 'RMSE for validation:  ', vali_err_4

RMSE for train:  0.908388858301
RMSE for validation:   1.05498628929


In [20]:
# learning_rate=0.01
W_5 = train_ridge_regression(train_ratings, X_n, n_users, learning_rate=0.05)
Y_pred_5 = X_n.dot(W_5) 
# tính độ lỗi
train_err_5 = RMSE(train_ratings, Y_pred_5)
vali_err_5 = RMSE(vali_ratings, Y_pred_5)
print 'RMSE for train: ', train_err_5
print 'RMSE for validation:  ', vali_err_5

RMSE for train:  0.906704260918
RMSE for validation:   1.06519460971


Ta thấy rằng, khi giảm learning_rate thì càng bị overfitting. Từ thử nghiệm trên ta thấy, learning_rate=0.5 cho kết quả tốt nhất trên validation. Ta sẽ sử dụng Y_pred_3 để tính độ lỗi trên test.

In [21]:
test_err = RMSE(test_ratings, Y_pred_3)
print 'RMSE for test: ', test_err

RMSE for test:  1.03246346043


## 6. Mô hình Matrix Factorization Collaborative Filtering

### a. Xây dựng mô hình

In [22]:
# Hàm chuẩn hóa dữ liệu theo user-based hoặc movie-based
def normalize_rating(Y, n_users, n_movies, user_based=1):
    """
    parameter:
        Y: ratings matrix chưa chuẩn hóa
        n_users: số lượng user
        n_movies: số lượng movie
        user_based: chuẩn hóa theo user (1) hay theo movie (0)
    return:
        Y_n: ratings matrix đã được chuẩn hóa
        mean_ratings: danh sách mean của từng vector trong ratings matrix. 
        Kích thước bằng n_users nếu chuẩn hóa theo user hoặc bằng n_movies nếu chuẩn hóa theo item
    """
    Y_n = Y.copy()
    if user_based: # user_based
        object_col = 0 # user_col
        n_objects = n_users
    else: # movie_based
        object_col = 1 # movie_col
        n_objects = n_movies

    objects = Y[:, object_col] 
    mean_rating = np.zeros((n_objects,))
    for n in xrange(n_objects):
        ids = np.where(objects == n)[0].astype(np.int32) # những hàng được rated bởi user n/ rate cho movie n
        ratings = Y_n[ids, 2] # và những rating tương ứng
        r_mean = np.mean(ratings) 
        if np.isnan(r_mean):
            r_mean = 0 # tránh giá trị NaN
        mean_rating[n] = r_mean
        Y_n[ids, 2] = ratings - mean_rating[n]
    return (Y_n, mean_rating) 

In [23]:
# Hàm lấy tất cả user đã rated movie và rating tương ứng
def get_users_who_rate_movie(ratings_matrix, movie_id):
    # ratings_matrix[:,1]: cột movie_id
    ids = np.where(ratings_matrix[:,1] == movie_id)[0] # tìm những hàng trong ratings rate cho movie
    user_ids = ratings_matrix[ids, 0].astype(np.int32) # users đã rated cho movie
    ratings = ratings_matrix[ids, 2] # và ratings tương ứng
    return (user_ids, ratings)

In [24]:
# Hàm tính giá trị mất mát
def loss(Y_n, X, W, lamda):
    n_ratings = Y_n.shape[0] # số lượng ratings
    L = 0 
    for i in xrange(Y_n.shape[0]):
        # user, movie, rating
        u, m, r = int(Y_n[i, 0]), int(Y_n[i, 1]), Y_n[i, 2]
        L += 0.5*(r - X[m, :].dot(W[:, u]))**2

    # regularization, don't ever forget this 
    L /= n_ratings
    L += 0.5*lamda*(np.linalg.norm(X, 'fro') + np.linalg.norm(W, 'fro'))
    return L

In [25]:
# Hàm update ma trận X
def updateX(Y_n, X, W, learning_rate, lamda):
    n_movies = X.shape[0] # số lượng movies
    n_ratings = Y_n.shape[0] # số lượng ratings
    K = X.shape[1]
    for m in xrange(n_movies):
        user_ids, ratings = get_users_who_rate_movie(Y_n, m)
        Wm = W[:, user_ids]
        grad_xm = -(ratings - X[m, :].dot(Wm)).dot(Wm.T)/n_ratings + lamda*X[m, :]
        X[m, :] -= learning_rate*grad_xm.reshape((K,))

In [26]:
# Hàm update ma trận W
def updateW(Y_n, X, W, learning_rate, lamda):
    n_users = W.shape[1] # số lượng user
    n_ratings = Y_n.shape[0] # số lượng ratings
    K = W.shape[0]
    for n in xrange(n_users):
        movie_ids, ratings = get_movies_rated_by_user(Y_n, n)
        Xn = X[movie_ids, :]
        grad_wn = -Xn.T.dot(ratings - Xn.dot(W[:, n]))/n_ratings + lamda*W[:, n]
        W[:, n] -= learning_rate*grad_wn.reshape((K,))

In [27]:
# hàm dự đoán ratings
def pred_ratings(X, W, avg_ratings, u, m):
    """ predict the rating of user u for movie m"""
    u = int(u)
    m = int(m)
    if avg_ratings.shape[0] == W.shape[1]:
        # vì W.shape[1] = n_users nên avg_ratings.shape[0] == W.shape[1] <=> user_based=1
        bias = avg_ratings[u]
    else: 
        # movie_based
        bias = avg_ratings[m]
    pred = X[m, :].dot(W[:, u]) + bias 
    return pred

In [28]:
# Hàm tính độ lỗi Root Mean Squared Error (RMSE)
def RMSE(Y, X, W, avg_ratings):
    n_ratings = Y.shape[0] # số lượng ratings
    SE = 0 # squared error
    for n in xrange(n_ratings):
        pred = pred_ratings(X, W, avg_ratings, Y[n, 0], Y[n, 1])
        SE += (pred - Y[n, 2])**2 
    RMSE = np.sqrt(SE/n_ratings)
    return RMSE

In [50]:
# Hàm huấn luyện sử dụng Gradient Descent
def train(Y_train, Y_vali, n_users, n_movies, K = 10, max_patience=None,
          learning_rate=0.5, lamda = 0.1, max_epoch = 100, user_based = 0):
    """
    parameter:
        Y_train: numpy array, train ratings
        Y_vali: numpy array, validation ratings
        n_users: int, number of users, 
        n_movies: int, number of movies
        K: int, number of latent features
        max_patience : int (> 0) or None
            The parameter of early stopping. We'll have a `patience` variable with initial value equal to
            `max_patience`. During the training, we'll keep track of the best MBE (Mean Binary Error) 
            on the validation set; if the MBE on the validation set at the current epoch < the current 
            best one, we'll reset `patience` to `max_patience`; otherwise, `patience` -= 1. 
            When `patience` = 0 or `max_epoch` is reached, we'll terminate SGD.
            If `max_patience` is None, we don't use early stopping.
        learning_rate : float, learning rate of SGD.
        lamda (weight decay level): float, the level (coefficient) of weight decay.
        max_epoch : int, we'll terminate SGD after this number of epochs or when `patience` = 0 (if early stopping is used).
    return:
        X, W
        avg_ratings: average ratings list (user based or item based)
    """
    Y_n, avg_rating = normalize_rating(Y_train, n_users, n_movies, user_based)
    # init X and W
    X = np.random.randn(n_movies, K)
    W = np.random.randn(K, n_users)
    # khởi tạo một số tham số
    best_X = X
    best_W = W
    best_epoch = -1 # ứng với vali_err bé nhất
    train_errs = []
    vali_errs = []
    min_vali_err = 100 # một số vừa đủ lớn, lưu giạ trị vali_err nhỏ nhất
    # train
    patience = max_patience
    epoch = 0
    for epoch in xrange(max_epoch):
        updateX(Y_n, X, W, learning_rate, lamda)
        updateW(Y_n, X, W, learning_rate, lamda)
        
        # compute train_err and vali_err
        train_err = RMSE(Y_train, X, W, avg_rating)
        vali_err = RMSE(Y_vali, X, W, avg_rating)
        train_errs.append(train_err)
        vali_errs.append(vali_err)
        # stop early
        if max_patience != None:
            if vali_err < min_vali_err:
                min_vali_err = vali_err          
                best_epoch = epoch
                best_W = W
                best_X = X
                patience = max_patience
            else:
                patience -= 1
            if patience == 0:
                break # break for epoch
    
    # print info of return best W and X
    if max_patience == None:
        print 'Info of returned: epoch ', epoch, ' train RMSE: ', train_errs[epoch], 'validation RMSE:', vali_errs[epoch]
        return (X, W, avg_rating)
    else:
        print 'Info of returned: epoch ', best_epoch, ' train RMSE: ', train_errs[best_epoch], 'validation RMSE:', vali_errs[best_epoch] 
        return (best_X, best_W, avg_rating)

### b. Huấn luyện mô hình

### TN1: so sánh ảnh hưởng của chuẩn hóa ma trận rating đến quá trình học.

Lần lượt thử với chuẩn hóa ratings theo user và movie. Cố định các giá trị K = 10, max_patience=None, learning_rate=0.5, lamda = 0.1, max_epoch = 100 (default).

In [30]:
# user-based
# n_users và n_movies đã được tính từ lúc trước khi chia dữ liệu
X_u, W_u, avg_rating_u = train(train_ratings, vali_ratings, n_users, n_movies, user_based=1)

Info of returned: epoch  99  train RMSE:  1.02578092137 validation RMSE: 1.03837239648


In [31]:
# movie-based
X_m, W_m, avg_rating_m = train(train_ratings, vali_ratings, n_users, n_movies, user_based=0)

C:\Anaconda2\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Anaconda2\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Info of returned: epoch  99  train RMSE:  0.974129869548 validation RMSE: 0.979579461814


**Nhận xét:** từ thử nghiệm trên ta thấy chuẩn hóa dữ liệu theo movie-based sẽ cho kết quả tốt hơn. Ở các thử nghiệm tiếp theo, ta sẽ mặc định sử dụng chuẩn hóa movie-based (user_based=0).

### TN2: ảnh hưởng của K đến quá trình học
Lần lượt thử nghiệm với K = 5, K = 30, K = 50. Cố định max_patience=None, learning_rate=0.5, lamda=0.1, max_epoch=100 (default).

In [32]:
# K = 5
X_1, W_1, avg_rating_1 = train(train_ratings, vali_ratings, n_users, n_movies, K = 5)

Info of returned: epoch  99  train RMSE:  0.974129980177 validation RMSE: 0.979579511178


In [33]:
# K = 30
X_2, W_2, avg_rating_2 = train(train_ratings, vali_ratings, n_users, n_movies, K = 30)

Info of returned: epoch  99  train RMSE:  0.974129860668 validation RMSE: 0.979579912528


In [34]:
# K = 50
X_3, W_3, avg_rating_3 = train(train_ratings, vali_ratings, n_users, n_movies, K = 50)

Info of returned: epoch  99  train RMSE:  0.974130272562 validation RMSE: 0.979579868461


**Nhận xét:** Khi tăng K quá lớn sẽ dẫn đến overfitting. Ở thí nghiệm này, giá trị K = 30 cho độ lỗi trên validation nhỏ nhất. Ta sẽ dùng K = 30 để huấn luyện và kiếm tra trên test.

##### TN3: Ảnh hưởng của learning_rate đến quá trình học
Thử nghiệm lần lượt learning_rate = 0.5, 0.1, 0.01. Cố định các giá trị k=10, max_patience=None, lamda=0.1, max_epoch=100 (default).

In [41]:
# learning_rate=0.5
X_4, W_4, avg_rating_4 = train(train_ratings, vali_ratings, n_users, n_movies, learning_rate=0.5)

Info of returned: epoch  99  train RMSE:  0.974130296693 validation RMSE: 0.979579868751


In [42]:
# learning_rate=0.1
X_5, W_5, avg_rating_5 = train(train_ratings, vali_ratings, n_users, n_movies, learning_rate=0.1)

Info of returned: epoch  99  train RMSE:  1.06025183522 validation RMSE: 1.06561020565


In [36]:
# learning_rate=0.01
X_6, W_6, avg_rating_6 = train(train_ratings, vali_ratings, n_users, n_movies, learning_rate=0.01)

Info of returned: epoch  99  train RMSE:  1.51415658309 validation RMSE: 1.51879106153


**Nhận xét:** khi giảm learning_rate, độ lỗi lại tăng trên cả tập train và validation. Giá trị learning_rate=0.5 cho kết quả tốt nhất, ta sẽ sử dụng để huấn luyện và kiểm tra trên test.

### TN4: ảnh hưởng của weight decay đến quá trình học.
Thử nghiệm lần lượt lamda = 0.0, 0.1. Cố định các giá trị k=10, max_patience=None, learning_rate=0.5, max_epoch=100 (default).

In [37]:
# lamda=0.0
X_7, W_7, avg_rating_7 = train(train_ratings, vali_ratings, n_users, n_movies, lamda=0.0)

Info of returned: epoch  99  train RMSE:  3.14772845478 validation RMSE: 3.1572188132


In [38]:
# lamda=0.1
X_8, W_8, avg_rating_8 = train(train_ratings, vali_ratings, n_users, n_movies, lamda=0.1)

Info of returned: epoch  99  train RMSE:  0.974130194555 validation RMSE: 0.979580354061


**Nhận xét:** khi sử dụng lamda>0, kết quả tốt hơn nhiều so với khi lamda=0.

### Kết quả dự đoán trên tập test:

In [52]:
X, W, avg_rating = train(train_ratings, vali_ratings, n_users, n_movies, K = 30, 
                         max_patience=50, learning_rate=0.5, lamda=0.1, max_epoch=200)

Info of returned: epoch  77  train RMSE:  0.962130172264 validation RMSE: 0.967280623471


In [53]:
test_err = RMSE(test_ratings, X, W, avg_rating)
print "Test RMSE: ", test_err

Test RMSE:  0.966832473621
